<a href="https://colab.research.google.com/github/ansonkwokth/PlackettLuceModel/blob/main/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !git clone https://github.com/ansonkwokth/PlackettLuceModel.git

In [2]:
# !python -m unittest plackett_luce/tests/test_utils.py

In [3]:
from plackett_luce import datasets as ds
from plackett_luce.model import PlackettLuceModel
from plackett_luce.utils import EarlyStopper

import numpy as np
import torch
from torch import nn

torch.manual_seed(0);


## Generate fake data with fixed number of items at each instance

In [19]:
# Parameters
num_samples_train = 5000
num_samples_test = 1000
num_items = 5

# Data generation
print("Generating training and testing data...")
X_train, rankings_train = ds.generate_data(num_samples_train, num_items)
X_test, rankings_test = ds.generate_data(num_samples_test, num_items)

# Create item masks for variable item counts
mask_train = torch.ones((num_samples_train, num_items)).int()
mask_test = torch.ones((num_samples_test, num_items)).int()

Generating training and testing data...


In [20]:
X_train.shape

torch.Size([5000, 5, 15])

### Define and train models

In [21]:
# Custom neural network model for flexible scoring
class NaiveNN(nn.Module):
    def __init__(self, input_dim):
        super(NaiveNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 1)  # 1D output for scoring
        )

    def forward(self, x):
        return self.network(x)





# Custom neural network model for flexible scoring
class LessNaiveNN(nn.Module):
    def __init__(self, input_dim):
        super(LessNaiveNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1)  # 1D output for scoring
        )

    def forward(self, x):
        return self.network(x)


In [22]:
num_features = X_train.shape[-1]

# Initialize the model
# custom_nn = NaiveNN(input_dim=num_features)
custom_nn = LessNaiveNN(input_dim=num_features)
# Custom early stopper
custom_early_stopper = EarlyStopper(patience=5, min_delta=0.01)
model = PlackettLuceModel(score_model=custom_nn, early_stopper=custom_early_stopper)
print(f"Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

# Training
print("Training the model...")

model.fit(X_train, rankings_train, lr=0.01, epochs=500, top_k=3, item_mask=mask_train)


Trainable params: 433
Training the model...
Epoch 10/500, Negative Log-Likelihood: 3.9073
Epoch 20/500, Negative Log-Likelihood: 3.4410
Epoch 30/500, Negative Log-Likelihood: 2.7851
Epoch 40/500, Negative Log-Likelihood: 2.1153
Epoch 50/500, Negative Log-Likelihood: 1.9149
Epoch 60/500, Negative Log-Likelihood: 1.8176
Epoch 70/500, Negative Log-Likelihood: 1.7612
Epoch 80/500, Negative Log-Likelihood: 1.7177
Early stopping at epoch 86 with NLL 1.6968


### Evaluate model

In [25]:
def evaluate(model, X_test, rankings_test):
    # Test the model
    print("\nTesting the model...\n")
    predicted_rankings = model.predict(X_test)

    # Evaluate the performance
    top1_correct = 0
    top2_correct = 0
    top3_correct = 0
    top1in3_correct = 0
    top2in3_correct = 0
    top1or2in3_correct = 0

    print_first_few = 10
    for i, (pred, true) in enumerate(zip(predicted_rankings, rankings_test.tolist())):
        if i < print_first_few:
            print(f"Sample {i + 1}:")
            print(f"  Predicted Ranking: {pred}")
            print(f"  True Ranking:      {true}")

        # Check Top-1 accuracy
        if pred[0] == true[0]:
            top1_correct += 1

        # Check Top-2 accuracy
        if pred[:2] == true[:2]:
            top2_correct += 1

        # Check Top-3 accuracy
        if pred[:3] == true[:3]:
            top3_correct += 1

        # Check Top-1 in first 3 accuracy
        if pred[0] in true[:3]:
            top1in3_correct += 1
        # Check Top-2 in first 3 accuracy
        if pred[1] in true[:3]:
            top2in3_correct += 1
        # Check Top-1 or 2 in first 3 accuracy
        if pred[0] in true[:3] or pred[1] in true[:3]:
            top1or2in3_correct += 1

    # Compute percentages
    top1_accuracy = top1_correct / X_test.shape[0] * 100
    top2_accuracy = top2_correct / X_test.shape[0] * 100
    top3_accuracy = top3_correct / X_test.shape[0] * 100
    top1in3_accuracy = top1in3_correct / X_test.shape[0] * 100
    top2in3_accuracy = top2in3_correct / X_test.shape[0] * 100
    top1or2in3_accuracy = top1or2in3_correct / X_test.shape[0] * 100

    print(f"\nTop-1 or 2 in 3 Accuracy: {top1or2in3_accuracy:.2f}%")
    print(f"Top-1 in 3 Accuracy: {top1in3_accuracy:.2f}%")
    print(f"Top-2 in 3 Accuracy: {top2in3_accuracy:.2f}%")
    print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
    print(f"Top-2 Accuracy: {top2_accuracy:.2f}%")
    print(f"Top-3 Accuracy: {top3_accuracy:.2f}%")


In [26]:
evaluate(model, X_test, rankings_test)


Testing the model...

Sample 1:
  Predicted Ranking: [4, 3, 1, 2, 0]
  True Ranking:      [4, 1, 3, 2, 0]
Sample 2:
  Predicted Ranking: [2, 0, 3, 4, 1]
  True Ranking:      [0, 2, 4, 3, 1]
Sample 3:
  Predicted Ranking: [3, 4, 1, 2, 0]
  True Ranking:      [3, 1, 4, 2, 0]
Sample 4:
  Predicted Ranking: [0, 2, 4, 3, 1]
  True Ranking:      [0, 2, 4, 3, 1]
Sample 5:
  Predicted Ranking: [0, 3, 4, 1, 2]
  True Ranking:      [0, 4, 2, 1, 3]
Sample 6:
  Predicted Ranking: [4, 1, 0, 3, 2]
  True Ranking:      [1, 4, 0, 3, 2]
Sample 7:
  Predicted Ranking: [2, 4, 1, 3, 0]
  True Ranking:      [2, 4, 1, 0, 3]
Sample 8:
  Predicted Ranking: [3, 1, 4, 0, 2]
  True Ranking:      [3, 1, 2, 4, 0]
Sample 9:
  Predicted Ranking: [0, 3, 4, 1, 2]
  True Ranking:      [0, 3, 4, 1, 2]
Sample 10:
  Predicted Ranking: [2, 1, 0, 4, 3]
  True Ranking:      [2, 1, 0, 4, 3]

Top-1 or 2 in 3 Accuracy: 100.00%
Top-1 in 3 Accuracy: 98.40%
Top-2 in 3 Accuracy: 93.90%
Top-1 Accuracy: 79.20%
Top-2 Accuracy: 58.80%

## Generate fake data with variable number of items at each instance

In [62]:
X_train_, rankings_train_ = ds.generate_data_varaible_items(num_samples=num_samples_train, num_items_range=(8, 14))
X_test_, rankings_test_ = ds.generate_data_varaible_items(num_samples=num_samples_test, num_items_range=(8, 14))




In [63]:
def find_min_max_items(rankings_train):
    min_items = min(len(rank_i) for rank_i in rankings_train)
    max_items = max(len(rank_i) for rank_i in rankings_train)

    return min_items, max_items


def get_masked(X, rankings, max_items):
    n_samples = len(rankings)
    n_features = len(X[0][0]) if X and X[0] else 0  # Handle empty input gracefully

    for i in range(n_samples):
        n_empty = max_items - len(X[i])
        if n_empty > 0:
            rankings[i].extend([np.nan] * n_empty)
            X[i].extend([[np.nan] * n_features] * n_empty)

    # Convert to numpy arrays
    X = np.array(X, dtype=float)
    rankings = np.array(rankings, dtype=float)

    # Create mask and handle NaNs
    mask = ~np.isnan(rankings)
    X[np.isnan(X)] = 0

    # Convert to PyTorch tensors
    X = torch.tensor(X, dtype=torch.float32)
    rankings = torch.tensor(rankings, dtype=torch.int32)
    mask = torch.tensor(mask, dtype=torch.int32)

    return X, rankings, mask


In [64]:
min_items, max_items = find_min_max_items(rankings_train_)

X_train, rankings_train, mask_train = get_masked(X_train_, rankings_train_, max_items)
X_test, rankings_test, mask_test = get_masked(X_test_, rankings_test_, max_items)


In [65]:

# Initialize the model
# custom_nn = NaiveNN(input_dim=num_features)
custom_nn = LessNaiveNN(input_dim=num_features)
# Custom early stopper
custom_early_stopper = EarlyStopper(patience=5, min_delta=0.01)
model = PlackettLuceModel(score_model=custom_nn, early_stopper=custom_early_stopper)
print(f"Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

# Training
print("Training the model...")

model.fit(X_train, rankings_train, lr=0.01, epochs=500, top_k=3, item_mask=mask_train)
# model.fit(X_train, rankings_train, lr=0.01, epochs=500)


Trainable params: 433
Training the model...
Epoch 10/500, Negative Log-Likelihood: 7.4823
Epoch 20/500, Negative Log-Likelihood: 6.9435
Epoch 30/500, Negative Log-Likelihood: 5.7595
Epoch 40/500, Negative Log-Likelihood: 4.5173
Epoch 50/500, Negative Log-Likelihood: 4.1411
Epoch 60/500, Negative Log-Likelihood: 3.9243
Epoch 70/500, Negative Log-Likelihood: 3.8142
Early stopping at epoch 79 with NLL 3.7444


In [66]:
evaluate(model, X_test, rankings_test)


Testing the model...

Sample 1:
  Predicted Ranking: [1, 3, 0, 4, 6, 7, 2, 5, 8, 9, 10, 11, 12, 13]
  True Ranking:      [1, 3, 0, 4, 7, 6, 5, 2, -2147483648, -2147483648, -2147483648, -2147483648, -2147483648, -2147483648]
Sample 2:
  Predicted Ranking: [8, 7, 2, 6, 1, 10, 4, 5, 3, 0, 9, 11, 12, 13]
  True Ranking:      [7, 8, 2, 6, 1, 10, 4, 5, 3, 9, 0, -2147483648, -2147483648, -2147483648]
Sample 3:
  Predicted Ranking: [6, 5, 2, 9, 3, 8, 1, 13, 11, 7, 4, 12, 10, 0]
  True Ranking:      [5, 2, 6, 9, 13, 3, 8, 1, 11, 4, 12, 10, 7, 0]
Sample 4:
  Predicted Ranking: [5, 8, 0, 3, 6, 9, 1, 7, 2, 10, 11, 12, 13, 4]
  True Ranking:      [5, 0, 8, 3, 6, 9, 7, 1, 2, 4, -2147483648, -2147483648, -2147483648, -2147483648]
Sample 5:
  Predicted Ranking: [2, 0, 3, 5, 7, 4, 6, 1, 8, 9, 10, 11, 12, 13]
  True Ranking:      [2, 0, 3, 7, 5, 4, 6, 1, -2147483648, -2147483648, -2147483648, -2147483648, -2147483648, -2147483648]
Sample 6:
  Predicted Ranking: [8, 2, 9, 7, 4, 5, 3, 6, 0, 1, 10, 11, 12